# Ensembling vgg16-Resnet50



In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten, Dense, Dropout, Average, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import tensorflow as tf

In [2]:
# Set dataset folder
dataset_folder = os.path.join(os.getcwd(), "parkinsons_dataset")
folders = os.listdir(dataset_folder)
print(folders)

['normal', 'parkinson']


In [3]:
# %% Load images and labels
# Define the updated label dictionary
label_dict = {
    'normal': 0,
    'parkinson': 1
}

In [4]:
# Function to load images from folder with reduced target size for memory optimization
def load_images_from_folder(folder_path, label_dict, target_size=(224, 224)):  # Changed target size to 224x224
    image_data = []
    labels = []
    folders = os.listdir(folder_path)
    
    for folder in folders:
        path = os.path.join(folder_path, folder)
        for im in os.listdir(path):
            img = image.load_img(os.path.join(path, im), target_size=target_size)
            img_array = image.img_to_array(img)
            image_data.append(img_array)
            labels.append(label_dict[folder])    
    return np.array(image_data), np.array(labels)

In [5]:
# Load the dataset
x_data, y_data = load_images_from_folder(dataset_folder, label_dict)

In [6]:
# %% Split the data into training (80%) and testing (20%) sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42, stratify=y_data)


In [7]:
# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

**Data augmentation**

In [8]:
# Data augmentation for training
augment = ImageDataGenerator(rotation_range=20,
                             width_shift_range=0.01,
                             height_shift_range=0.01,
                             horizontal_flip=False,
                             vertical_flip=False)
augment.fit(x_train)

In [9]:
# %% Model 1: VGG16
# Load the VGG16 model and freeze more layers
vgg16_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))  # Adjusted input shape
for layer in vgg16_model.layers[:12]:  # Freeze more layers to optimize memory
    layer.trainable = False

vgg16_output = Flatten()(vgg16_model.output)
vgg16_fc1 = Dense(256, activation='relu')(vgg16_output)
vgg16_dropout1 = Dropout(0.5)(vgg16_fc1)
vgg16_fc2 = Dense(128, activation='relu')(vgg16_dropout1)
vgg16_dropout2 = Dropout(0.5)(vgg16_fc2)
vgg16_output_layer = Dense(2, activation='sigmoid')(vgg16_dropout2)
model_vgg = Model(vgg16_model.input, vgg16_output_layer)

In [10]:
# %% Model 2: ResNet50
# Load the ResNet50 model and freeze more layers
resnet50_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))  # Adjusted input shape
for layer in resnet50_model.layers[:150]:  # Freeze more layers to optimize memory
    layer.trainable = False

resnet_output = Flatten()(resnet50_model.output)
resnet_fc1 = Dense(256, activation='relu')(resnet_output)
resnet_dropout1 = Dropout(0.5)(resnet_fc1)
resnet_fc2 = Dense(128, activation='relu')(resnet_dropout1)
resnet_dropout2 = Dropout(0.5)(resnet_fc2)
resnet_output_layer = Dense(2, activation='sigmoid')(resnet_dropout2)
model_resnet = Model(resnet50_model.input, resnet_output_layer)

In [11]:
# %% Ensemble the models by concatenating their outputs for better performance
model_input = tf.keras.Input(shape=(224, 224, 3))  # Adjusted input shape
vgg_output = model_vgg(model_input)
resnet_output = model_resnet(model_input)
ensemble_output = Concatenate()([vgg_output, resnet_output])  # Concatenate instead of average for more information
final_output = Dense(2, activation='sigmoid')(ensemble_output)  # Final output layer
ensemble_model = Model(inputs=model_input, outputs=final_output)

In [12]:
# %% Compile the ensemble model
adam = Adam(learning_rate=0.0001)
ensemble_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


In [13]:
# Save model checkpoint
checkpoint_filepath = "parkinsons_detection_ensemble.hdf5"
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', verbose=2, save_best_only=True, mode='min', patience=5)
callbacks_list = [checkpoint]

In [14]:
# %% Train the model with reduced batch size
hist = ensemble_model.fit(augment.flow(x_train, y_train, batch_size=16),  # Reduced batch size to 16
                          epochs=20,
                          validation_data=(x_test, y_test),
                          callbacks=callbacks_list)

Epoch 1/20
10/10 [==============================] - ETA: 0s - loss: 0.7462 - accuracy: 0.6750 
Epoch 1: val_loss improved from inf to 0.44271, saving model to parkinsons_detection_ensemble.hdf5
10/10 [==============================] - 671s 68s/step - loss: 0.7462 - accuracy: 0.6750 - val_loss: 0.4427 - val_accuracy: 0.9250
Epoch 2/20
10/10 [==============================] - ETA: 0s - loss: 0.6816 - accuracy: 0.7375  
Epoch 2: val_loss did not improve from 0.44271
10/10 [==============================] - 677s 70s/step - loss: 0.6816 - accuracy: 0.7375 - val_loss: 0.5364 - val_accuracy: 0.9500
Epoch 3/20
10/10 [==============================] - ETA: 0s - loss: 0.6151 - accuracy: 0.8500 
Epoch 3: val_loss did not improve from 0.44271
10/10 [==============================] - 667s 68s/step - loss: 0.6151 - accuracy: 0.8500 - val_loss: 0.5617 - val_accuracy: 0.9250
Epoch 4/20
10/10 [==============================] - ETA: 0s - loss: 0.6173 - accuracy: 0.8625  
Epoch 4: val_loss did not improv

In [16]:
# Load the best saved model
best_model = load_model(checkpoint_filepath)

In [17]:
# %% Evaluate the model on the test data
test_loss, test_accuracy = best_model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

2/2 [==============================] - 62s 14s/step - loss: 0.4427 - accuracy: 0.9250
Test Loss: 0.44271135330200195, Test Accuracy: 0.925000011920929


In [18]:
# %% Predict test data
predictions = best_model.predict(x_test)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)

2/2 [==============================] - 71s 15s/step


In [19]:
# %% Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

# Print classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))

print("Confusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

Accuracy: 0.925
Precision: 0.875
Recall: 1.0
F1-score: 0.9333333333333333

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.84      0.91        19
           1       0.88      1.00      0.93        21

    accuracy                           0.93        40
   macro avg       0.94      0.92      0.92        40
weighted avg       0.93      0.93      0.92        40

Confusion Matrix:
[[16  3]
 [ 0 21]]
